In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import shapely as sp
from shapely.geometry import Point, LineString, Polygon

In [2]:
zipcodes = gpd.read_file('../data/clean/zipcodes.geojson')
zipcodes = zipcodes[zipcodes['zip'].isin(['37206', '37216', '37115'])]
zipcodes
center = zipcodes.geometry.centroid[1]
map_center = [center.y, center.x]

C:\Users\chabi\AppData\Local\Temp\ipykernel_22000\598183039.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = zipcodes.geometry.centroid[1]


In [3]:
crashes = pd.read_csv('../data/clean/crashes.csv')
crashes = crashes[['accident_number', 'lat', 'long']]
crashes_geo = gpd.GeoDataFrame(crashes, crs= zipcodes.crs, geometry=gpd.points_from_xy(crashes.long, crashes.lat))

Create a heat map of all the crashes in the study area.

In [4]:
all_crashes = folium.Map(location = map_center, tiles="Cartodb Positron", zoom_start = 12)

folium.GeoJson(zipcodes, style_function=lambda feature: {"color": "black", "weight": 2, "dashArray": "10, 5","fillOpacity":0.125}).add_to(all_crashes)

crashes = []
for row_index, row_values in crashes_geo.iterrows():
    loc = [row_values['lat'], row_values['long']]
    crashes.append(loc)
crashes

HeatMap(crashes, radius = 15, min_opacity = 0.5, gradient={.5: '#ffc2c2', .75: '#ff7970', 1: '#ff0000'}).add_to(folium.FeatureGroup(name='Heat Map').add_to(all_crashes))
folium.LayerControl().add_to(all_crashes)

all_crashes.save('../maps/all_crashes.html')

Updating the map to differentiate collision types.

Now let's see where there are 311 requests along the corridor.

In [5]:
nash_311 = pd.read_csv('../data/clean/nash_311.csv')

In [6]:
nash_311_geo = nash_311[['case_number', 'latitude', 'longitude']]
nash_311_geo = gpd.GeoDataFrame(nash_311, crs= zipcodes.crs, geometry=gpd.points_from_xy(nash_311.longitude, nash_311.latitude))

In [7]:
nash_311 = folium.Map(location = map_center, tiles="Cartodb Positron", zoom_start = 12)

folium.GeoJson(zipcodes, style_function=lambda feature: {"color": "black", "weight": 2, "dashArray": "10, 5","fillOpacity":0.125}).add_to(nash_311)

requests_311 = []
for row_index, row_values in nash_311_geo.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    requests_311.append(loc)
requests_311

HeatMap(requests_311, radius = 15, min_opacity = 0.5, gradient={.5: '#98ABEE', .75: '#1D24CA', 1: '#201658'}).add_to(folium.FeatureGroup(name='Heat Map').add_to(nash_311))
folium.LayerControl().add_to(nash_311)

nash_311.save('../maps/nash_311.html')

Same for short-term rental permits.

In [8]:
rentals = pd.read_csv('../data/clean/rentals.csv')

In [9]:
rentals['mapped_location'][0]

'{\'latitude\': \'36.18643801\', \'longitude\': \'-86.727935964\', \'human_address\': \'{"address": "21 WATERS AVE", "city": "NASHVILLE", "state": "TN", "zip": "37206"}\'}'

At a high level, there seems to be at least some correlation between frequency of crashes and frequency of 311 calls. Next, back to the EDA notebook to look at crash types and 311 categories.

In [10]:
impr_requests = pd.read_csv('../data/clean/impr_requests.csv')
impr_requests = impr_requests[['case_number', 'latitude', 'longitude']]
requests_311 = gpd.GeoDataFrame(impr_requests, crs= zipcodes.crs, geometry=gpd.points_from_xy(impr_requests.longitude, impr_requests.latitude))

In [11]:
req_311 = folium.Map(location = map_center, tiles="Cartodb Positron", zoom_start = 12)

folium.GeoJson(zipcodes, style_function=lambda feature: {"color": "black", "weight": 2, "dashArray": "10, 5","fillOpacity":0.125}).add_to(req_311)

requests = []
for row_index, row_values in requests_311.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    requests.append(loc)
requests

HeatMap(requests, radius = 15, min_opacity = 0.5, gradient={.5: '#ffc2c2', .75: '#ff7970', 1: '#ff0000'}).add_to(folium.FeatureGroup(name='Heat Map').add_to(req_311))
folium.LayerControl().add_to(req_311)

all_crashes.save('../maps/req_311.html')